In [1]:
#!g1.1
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
#!g1.1
from random import random

from SourceCodeTools.models.training_config import get_config, save_config, load_config
from SourceCodeTools.code.data.dataset.Dataset import SourceGraphDataset, filter_dst_by_freq
from SourceCodeTools.models.graph.train.sampling_multitask2 import training_procedure, SamplingMultitaskTrainer
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import NodeClassifierObjective
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective
from SourceCodeTools.models.graph.train.utils import get_name, get_model_base
from SourceCodeTools.models.graph import RGGAN
from SourceCodeTools.tabular.common import compact_property
from SourceCodeTools.code.data.file_utils import unpersist

import dgl
import torch
import numpy as np
from argparse import Namespace
from torch import nn
from datetime import datetime
from os.path import join
from functools import partial

Using backend: pytorch


# Prepare parameters and options

Full list of options that can be added can be found in `SourceCodeTools/models/training_options.py`. They are ment to be used as arguments for cli trainer. Trainer script can be found in `SourceCodeTools/scripts/train.py`.

For the task of subgraph classification the important options are:
- `subgraph_partition` is path to subgraph-based train/val/test sets. Storead as Dataframe with subgraph id and partition mask
- `subgraph_id_column` is a column is `common_edges` file that stores subgraph id.
- For variable misuse task (same will apply to authorship attribution) subgraphs are created for individual functions (files for SCAA). The label is stored in `common_filecontent`.

In [3]:
#!g1.1
tokenizer_path = "sentencepiece_bpe.model"

data_path = "10_percent_v1"
subgraph_partition = join(data_path, "partition.json.bz2")
filecontent_path = join(data_path, "common_filecontent.json.bz2")

# Data distribution

In [4]:
#!g1.1
labels = unpersist(filecontent_path)
for part in labels['partition'].unique():
    print(part)
    print(labels.query('partition == @part')['label'].value_counts())
    print()

train
Correct            35036
Variable misuse    35036
Name: label, dtype: int64

dev
Correct            3754
Variable misuse    3754
Name: label, dtype: int64

eval
Correct            18787
Variable misuse    18787
Name: label, dtype: int64



In [5]:
#!g1.1
unpersist(join(data_path, "common_edges.json.bz2"), nrows=10)

,id,type,source_node_id,target_node_id,file_id,mentioned_in,offset_start,offset_end
0,0,subword,4681713,2314503,6868,NaN,NaN,NaN
1,1,arg,2314503,3039950,6868,2388425.0,NaN,NaN
2,2,arg_rev,3039950,2314503,6868,2388425.0,NaN,NaN
3,3,args,3039950,1758356,6868,2388425.0,13.0,17.0
4,4,args_rev,1758356,3039950,6868,2388425.0,NaN,NaN
5,5,subword,1045672,2068664,6868,NaN,NaN,NaN
6,6,arg,2068664,436348,6868,2388425.0,NaN,NaN
7,7,arg_rev,436348,2068664,6868,2388425.0,NaN,NaN
8,8,args,436348,1758356,6868,2388425.0,19.0,25.0
9,9,args_rev,1758356,436348,6868,2388425.0,NaN,NaN


# Config

In [6]:
#!g1.1
config = get_config(
    subgraph_id_column="file_id",
    data_path=data_path,
    model_output_dir=data_path,
    subgraph_partition=subgraph_partition,
    tokenizer_path=tokenizer_path,
    objectives="subgraph_clf",
    #use_edge_types=True,
    gpu=-1,
    epochs=3, 
    
    train_frac=0.8,                      # *** fraction of nodes to use for training
    random_seed=42, 
    
    # model parameters
#     elem_emb_size=200,
#     node_emb_size=200,                   # *** dimensionality of node embeddings
#     h_dim=200,                           # *** should match to node dimensionality
    n_layers=2,
#     num_bases=10,                        # number of bases for computing parmetwer weights for different edge types
#     dropout=0.2,                         # *** 
#     use_self_loop=True,                  #
#     activation="tanh",                   # *** 
#     learning_rate=3e-2,                  # *** 
#     use_gcn_checkpoint=True,
#     use_att_checkpoint=True,
#     use_gru_checkpoint=True
)

In [7]:
#!g1.1
save_config(config, "var_misuse_tiny.yaml")

In [8]:
#!g1.1
config

{'DATASET': {'data_path': '10_percent_v1',
  'train_frac': 0.8,
  'filter_edges': None,
  'min_count_for_objectives': 5,
  'self_loops': False,
  'use_node_types': False,
  'use_edge_types': False,
  'no_global_edges': False,
  'remove_reverse': False,
  'custom_reverse': None,
  'restricted_id_pool': None,
  'random_seed': 42,
  'subgraph_id_column': 'file_id',
  'subgraph_partition': '10_percent_v1\\partition.json.bz2'},
 'TRAINING': {'model_output_dir': '10_percent_v1',
  'pretrained': None,
  'pretraining_phase': 0,
  'sampling_neighbourhood_size': 10,
  'neg_sampling_factor': 3,
  'use_layer_scheduling': False,
  'schedule_layers_every': 10,
  'elem_emb_size': 100,
  'embedding_table_size': 200000,
  'epochs': 10,
  'batch_size': 128,
  'learning_rate': 0.001,
  'objectives': 'subgraph_clf',
  'save_each_epoch': False,
  'save_checkpoints': True,
  'early_stopping': False,
  'early_stopping_tolerance': 20,
  'force_w2v_ns': False,
  'use_ns_groups': False,
  'nn_index': 'brute',
 

# Create Dataset

In [9]:
#!g1.1
dataset = SourceGraphDataset(
    **{**config["DATASET"], **config["TOKENIZER"]}
)

# Declare target loading function (labels)

In [10]:
#!g1.1
def load_labels():
    filecontent = unpersist(filecontent_path)
    return filecontent[["id", "label"]].rename({"id": "src", "label": "dst"}, axis=1)

One or several objectives could be used

In [11]:
#!g1.1
class Trainer(SamplingMultitaskTrainer):
    def create_objectives(self, dataset, tokenizer_path):
        self.objectives = nn.ModuleList()
        
        self.objectives.append(
            self._create_subgraph_objective(
                objective_name="VariableMisuseClf",
                objective_class=SubgraphClassifierObjective,
                dataset=dataset,
                tokenizer_path=tokenizer_path,
                labels_fn=load_labels,
            )
        )

In [12]:
#!g1.1
data_path

'10_percent_v1'

In [14]:
#!g1.1
%tensorboard --logdir './10_percent_v1'

In [13]:
#!g1.1
training_procedure(
    dataset, 
    model_name=RGGAN, 
    model_params=config["MODEL"],
    trainer_params=config["TRAINING"],
    model_base_path=get_model_base(config["TRAINING"], get_name(RGGAN, str(datetime.now()))),
    trainer=Trainer
)

Number of nodes 4900346


Epoch 1:   0%|                                                                                 | 0/548 [00:00<?, ?it/s]

Epoch: 0, Time: 1143 s
{'Accuracy/test/VariableMisuseClf_': 0.5943027210884354,
 'Accuracy/train/VariableMisuseClf_': 0.5892857142857143,
 'Accuracy/train_avg/VariableMisuseClf': 0.5570031445516163,
 'Accuracy/val/VariableMisuseClf_': 0.5972117130750605,
 'Loss/test/VariableMisuseClf_': 0.6575186990556263,
 'Loss/train/VariableMisuseClf_': 0.6637745499610901,
 'Loss/train_avg/VariableMisuseClf': 0.6783770066108147,
 'Loss/val/VariableMisuseClf_': 0.6569047287359076,
 'grad_norm/train/_': 0.42949146926510684}


Epoch 2:   0%|                                                                                 | 0/548 [00:00<?, ?it/s]

Epoch: 1, Time: 1124 s
{'Accuracy/test/VariableMisuseClf_': 0.598430666909621,
 'Accuracy/train/VariableMisuseClf_': 0.5714285714285714,
 'Accuracy/train_avg/VariableMisuseClf': 0.6101896506777893,
 'Accuracy/val/VariableMisuseClf_': 0.6048287429378532,
 'Loss/test/VariableMisuseClf_': 0.6477953039464497,
 'Loss/train/VariableMisuseClf_': 0.6415787935256958,
 'Loss/train_avg/VariableMisuseClf': 0.6419532827652283,
 'Loss/val/VariableMisuseClf_': 0.6449324377512528,
 'grad_norm/train/_': 0.528024339020724}


...

Epoch 9:   0%|                                                                                 | 0/548 [00:00<?, ?it/s]

Epoch: 8, Time: 1125 s
{'Accuracy/test/VariableMisuseClf_': 0.6412566812439261,
 'Accuracy/train/VariableMisuseClf_': 0.6607142857142857,
 'Accuracy/train_avg/VariableMisuseClf': 0.698186180265902,
 'Accuracy/val/VariableMisuseClf_': 0.6460855528652139,
 'Loss/test/VariableMisuseClf_': 0.6275826931810703,
 'Loss/train/VariableMisuseClf_': 0.5460851788520813,
 'Loss/train_avg/VariableMisuseClf': 0.5382743280408157,
 'Loss/val/VariableMisuseClf_': 0.618916322619228,
 'grad_norm/train/_': 1.4791041828474847}


Epoch 9: 100%|███████████████████████████████████████████████████████████████████████| 548/548 [12:29<00:00,  1.37s/it]


Epoch: 9, Time: 1119 s
{'Accuracy/test/VariableMisuseClf_': 0.6495520529640428,
 'Accuracy/train/VariableMisuseClf_': 0.6785714285714286,
 'Accuracy/train_avg/VariableMisuseClf': 0.7032655272419187,
 'Accuracy/val/VariableMisuseClf_': 0.6527693704600483,
 'Loss/test/VariableMisuseClf_': 0.6209840936725642,
 'Loss/train/VariableMisuseClf_': 0.5306960344314575,
 'Loss/train_avg/VariableMisuseClf': 0.5293318730418699,
 'Loss/val/VariableMisuseClf_': 0.6123148334228387,
 'grad_norm/train/_': 1.0313846618034506}
Final eval: Loss/val/VariableMisuseClf_final: 0.6123148334228387, Accuracy/val/VariableMisuseClf_final: 0.6527693704600483, Loss/test/VariableMisuseClf_final: 0.6209840936725642, Accuracy/test/VariableMisuseClf_final: 0.6495520529640428


(<__main__.Trainer at 0x26fb3578d60>,
 {'Loss/val/VariableMisuseClf_final': 0.6123148334228387,
  'Accuracy/val/VariableMisuseClf_final': 0.6527693704600483,
  'Loss/test/VariableMisuseClf_final': 0.6209840936725642,
  'Accuracy/test/VariableMisuseClf_final': 0.6495520529640428})

In [15]:
#!g1.1


Error: Not enough permissions to perform requested action

In [16]:
#!g1.1


Error: Not enough permissions to perform requested action